Import Dependencies

In [8]:
import pandas as pd
import numpy as np
from collections import Counter

In [7]:
data = {
    'income' : ['high', 'high', 'high', 'medium', 'low', 'low', 'low', 'medium','low', 'medium', 'medium', 'medium', 'high', 'medium'],
    'student' : ['no','no','no','no','yes','yes','yes','no','yes','yes','yes','no','yes','no'],
    'age' : [30 , 27 , 45 , 58 , 20 , 23  , 22 , 46 , 23 , 25 , 22 , 36 , 20 , 40  ] , 
    'credit_rate' : ['fair', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'fair', 'fair','excellent', 'excellent', 'fair', 'excellent'],
    'Outcome' : ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes','yes', 'yes', 'yes', 'no']
}
data = pd.DataFrame (data, columns=data.keys())
print(data)


    income student  age credit_rate Outcome
0     high      no   30        fair      no
1     high      no   27   excellent      no
2     high      no   45        fair     yes
3   medium      no   58        fair     yes
4      low     yes   20        fair     yes
5      low     yes   23   excellent      no
6      low     yes   22   excellent     yes
7   medium      no   46        fair      no
8      low     yes   23        fair     yes
9   medium     yes   25        fair     yes
10  medium     yes   22   excellent     yes
11  medium      no   36   excellent     yes
12    high     yes   20        fair     yes
13  medium      no   40   excellent      no


In [26]:
result_class = 'Outcome'

In [46]:
number_of_classes = len(Counter(data[result_class]))
classes_items = list(Counter(data[result_class]).keys())

In [33]:
print("Number of Classes Present is : ", number_of_classes)

Number of Classes Present is :  2


In [49]:
print("The Classes for prediction are " , classes_items)

The Classes for prediction are  ['no', 'yes']


In [129]:
def gini_cat( feature , dataset = data  , n_class =  number_of_classes , class_item =  classes_items , result = result_class ):
    feature_data = list(dataset[feature])
    items = list(Counter(feature_data).keys())
    n = len(dataset)
    probability_sum_sq = 0
    for i in items:
        probability_sum_sq+=(len(dataset[dataset[feature] == i]) / n)**2
    return (1 - probability_sum_sq)

In [140]:
categorical = list(data.iloc[: , :-1].select_dtypes(exclude = ['int' , 'float']).columns)
numerical = list(data.iloc[: , :-1].select_dtypes(include = ['int' , 'float']).columns)

In [141]:
categorical

['income', 'student', 'credit_rate']

In [142]:
numerical

['age']

In [143]:
gini = {}

In [144]:
for category in categorical:
    gini[category] = gini_cat(category)

In [145]:
gini

{'income': 0.653061224489796,
 'student': 0.5,
 'credit_rate': 0.48979591836734704}

In [194]:
def gini_num(feature , dataset = data  , n_class =  number_of_classes , class_item =  classes_items , result = result_class):
    sorted_data = dataset.sort_values(by = feature)
    sorted_col = sorted(list(set(sorted_data[feature])))
    l = []
    for i in sorted_col:
        left = sorted_data[sorted_data[feature]<=i]
        prob = []
        left_len = len(left)
        for class_ in class_item:
            prob.append(len(left[left[result] == class_]))
        l.append(prob)
    for i in range(len(l)):
        s = sum(l[i])
        new_l = []
        for j in l[i]:
            new_l.append(j/s)
        l[i] = new_l
    gini_l = []
    for i in l:
        s = 0
        for j in i:
            s+=(j**2)
        gini_l.append(1-s)
    l = []
    for i in sorted_col:
        right = sorted_data[sorted_data[feature]>i]
        prob = []
        right_len = len(right)
        for class_ in class_item:
            prob.append(len(left[left[result] == class_]))
        l.append(prob)
    for i in range(len(l)):
        s = sum(l[i])
        new_l = []
        for j in l[i]:
            new_l.append(j/s)
        l[i] = new_l
    gini_r = []
    for i in l:
        s = 0
        for j in i:
            s+=(j**2)
        gini_r.append(1-s)
    n = len(dataset)
    gini = []
    for i in range(len(sorted_col)):
        p_l = len(dataset[dataset[feature]<=sorted_col[i]])/n
        p_r = len(dataset[dataset[feature]>sorted_col[i]])/n
        gini.append( (p_l*gini_l[i]) + (p_r*gini_r[i]))
    index = gini.index(min(gini))
    return (sorted_col[index] , gini[index])

In [195]:
gini_num('age')

(22, 0.3279883381924198)

In [203]:
for feature in numerical:
    gini[feature] = gini_num(feature)[1]

In [204]:
gini

{'income': 0.653061224489796,
 'student': 0.5,
 'credit_rate': 0.48979591836734704,
 'age': 0.3279883381924198}

In [205]:
print("Best Feature is :  " , min(gini))

Best Feature is :   age


In [206]:
if min(gini) in  numerical:
    val , feature = gini_num(min(gini))